In [1]:
import os
import numpy as np
import trimesh
import pandas as pd
import vtk
import logging

# Extracting the geometrical features of every mesh from the 4 runs with different hyperparameters

Here I will extract the features which were present in the [original Thingi10K dataset geometrical descriptions](https://docs.google.com/spreadsheets/d/1ZM5_1ry3Oe5uDJZxQIcFR6fjjas5rX4yjkhQ8p7Kf2Q/edit?gid=1531775051#gid=1531775051).

A lot of the time I didn't have a good solution on how to extract some of the features, so, because I hadn't the time to figure it out, I used AI models to hint me in the right direction.

## Note: there is a cell that runs more than 40 minutes. Run at your own risk.

In [2]:
def calculate_num_vertices(mesh):
    """
    Calculate the number of vertices in the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The number of vertices in the mesh.
    """
    return len(mesh.vertices)

In [3]:
def calculate_num_faces(mesh):
    """
    Calculate the number of faces in the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The number of faces in the mesh.
    """
    return len(mesh.faces)

In [4]:
def calculate_num_geometrical_degenerated_faces(mesh, threshold=1e-12):
    """
    Calculate the number of geometrically degenerated faces in the mesh.

    Geometrically degenerated faces are faces with an area smaller than a specified threshold.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.
    threshold (float): The area threshold below which faces are considered degenerated. Default is 1e-12.

    Returns:
    int: The number of geometrically degenerated faces in the mesh.
    """
    return np.sum(mesh.area_faces < threshold)

In [5]:
def calculate_num_combinatorial_degenerated_faces(mesh):
    """
    Calculate the number of combinatorial degenerated faces in the mesh.

    Combinatorial degenerated faces are faces that are duplicated within the mesh.
    This method identifies such faces by sorting the vertices of each face and
    checking for duplicates.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The number of combinatorial degenerated faces in the mesh.
    """
    sorted_faces = np.sort(mesh.faces, axis=1)
    _, face_counts = np.unique(sorted_faces, axis=0, return_counts=True)
    return np.sum(face_counts > 1)

In [6]:
def calculate_num_duplicated_faces(mesh):
    """
    Calculate the number of exact duplicated faces in the mesh.

    This method identifies and counts faces that are exactly identical within the mesh, 
    meaning they have the same vertices in the same order.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The number of exact duplicated faces in the mesh.
    """
    _, exact_face_counts = np.unique(mesh.faces, axis=0, return_counts=True)
    return np.sum(exact_face_counts > 1)

In [7]:
def calculate_num_connected_components(mesh):
    """
    Calculate the number of connected components in the mesh.

    Connected components are distinct parts of the mesh that are not connected to each other.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The number of connected components in the mesh.
    """
    return len(mesh.split(only_watertight=False))

In [8]:
def calculate_num_boundary_edges(mesh):
    """
    Calculate the number of boundary edges in the mesh.

    A boundary edge is an edge that belongs to exactly one face.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    int: The number of boundary edges.
    """
     # Get the sparse matrix of edges to faces and convert to dense
    edges_sparse = mesh.edges_sparse.tocoo()

    # Count how many times each edge is referenced by a face
    edge_face_counts = np.bincount(edges_sparse.row, minlength=mesh.edges.shape[0])
    
    num_boundary_edges = np.sum(edge_face_counts == 1)
    return num_boundary_edges

In [9]:
def calculate_euler_characteristic(mesh):
    """
    Calculate the Euler characteristic of the mesh.

    The Euler characteristic is a topological invariant that represents the shape of the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The Euler characteristic of the mesh.
    """
    return mesh.euler_number

In [10]:
def calculate_num_self_intersections(mesh):
    """
    Estimate the number of self-intersections in the mesh.

    Self-intersections are parts of the mesh where the surface intersects itself.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    int: The estimated number of self-intersections in the mesh.
    """
    # Attempt to estimate the number of self-intersections in a mesh by slicing it with a plane (parallel to the YZ plane in this case) and passes through the origin.
    # The idea is that if the mesh has self-intersections, slicing it with a plane should produce line segments or intersection points where the mesh intersects itself.
    intersections = trimesh.intersections.mesh_plane(mesh, [1, 0, 0], [0, 0, 0])
    return len(intersections) if intersections is not None else 0

In [11]:
def calculate_num_coplanar_intersecting_faces(mesh, tolerance=1e-5):
    """
    Calculate the number of coplanar intersecting faces in the mesh.
    
    Two faces are considered coplanar if they lie on the same plane.
    In terms of geometry, this means that the normals of the two faces are either exactly parallel (or anti-parallel).

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.
    tolerance (float): The tolerance for determining if faces are coplanar.

    Returns:
    int: The number of coplanar intersecting face pairs.
    """
    face_normals = mesh.face_normals
    adjacency = mesh.face_adjacency
    coplanar_count = 0

    for adj in adjacency:
        normal_1 = face_normals[adj[0]]
        normal_2 = face_normals[adj[1]]
        
        # Check if the dot product of normals is close to 1 or -1 (parallel (or anti-parallel) vectors)
        if np.abs(np.dot(normal_1, normal_2)) > 1 - tolerance:
            coplanar_count += 1

    return coplanar_count

In [12]:
def check_vertex_manifold(mesh):
    """
    Checks if the mesh is vertex manifold.

    A vertex is manifold if all edges connected to it form a fan, meaning the
    faces connected to the vertex form a contiguous, non-overlapping set.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    bool: True if the mesh is vertex manifold, False otherwise.
    """
    for vertex_id in range(len(mesh.vertices)):
        adjacent_faces = mesh.vertex_faces[vertex_id]
        adjacent_faces = adjacent_faces[adjacent_faces != -1]
        if len(adjacent_faces) < 3:
            continue
        # Check if the adjacent faces form a fan
        fan_edges = np.unique(mesh.faces[adjacent_faces].ravel())
        if len(fan_edges) != len(adjacent_faces) + 1:
            return False
    return True

In [13]:
def check_edge_manifold(mesh):
    """
    Checks if the mesh is edge manifold.

    An edge is manifold if it is shared by exactly two faces.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    bool: True if the mesh is edge manifold, False otherwise.
    """
    edge_faces = mesh.edges_face

    # Handle the case where edge_faces is not a 2D array (non-manifold edges, degenerate or broken meshes, single face meshes)
    if edge_faces.ndim == 1:
        # Single face case
        edge_face_counts = np.bincount(edge_faces, minlength=len(mesh.edges))
    else:
        # General case
        edge_face_counts = np.count_nonzero(edge_faces != -1, axis=1)

    # Check if all edges are connected to exactly two faces
    return np.all(edge_face_counts == 2)

In [14]:
def check_oriented(mesh):
    """
    Checks if the mesh is consistently oriented.

    A mesh is oriented if all face normals are consistently pointing in the
    correct relative direction.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    bool: True if the mesh is oriented, False otherwise.
    """
    face_normals = mesh.face_normals
    adjacency = mesh.face_adjacency
    for adj in adjacency:
        if np.dot(face_normals[adj[0]], face_normals[adj[1]]) < 0:
            # The vectors are parallel but pointing in opposite directions (anti-parallel).
            return False
    return True

In [15]:
def calculate_vertex_valence(mesh):
    """
    Calculate the valence (degree) of each vertex in a 3D mesh.

    The vertex valence is defined as the number of edges connected to a vertex. This method computes
    the valence for each vertex in the mesh and returns various statistical measures of the vertex valences.
    (The valence of a vertex is calculated by counting the number of edges that include the vertex.)

    Parameters:
    mesh (trimesh.Trimesh): The input mesh represented as a `trimesh.Trimesh` object.

    Returns:
    dict: A dictionary containing various statistics about the vertex valences.
    """
    vertex_valence = np.bincount(mesh.edges_unique.ravel(), minlength=len(mesh.vertices))
    return {
        'min_valance': np.min(vertex_valence),
        'p25_valance': np.percentile(vertex_valence, 25),
        'median_valance': np.median(vertex_valence),
        'p75_valance': np.percentile(vertex_valence, 75),
        'p90_valance': np.percentile(vertex_valence, 90),
        'p95_valance': np.percentile(vertex_valence, 95),
        'max_valance': np.max(vertex_valence),
        'ave_valance': np.mean(vertex_valence),
    }

In [16]:
def calculate_aspect_ratios_vtk(mesh):
    """
    Calculate the aspect ratios of triangular faces in a 3D mesh using VTK.

    The aspect ratio of a triangle is defined as the ratio of the longest edge to the shortest altitude. 
    This method utilizes the VTK library to compute aspect ratios for all triangular faces in the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh represented as a `trimesh.Trimesh` object. The mesh should consist of triangular faces.

    Returns:
    dict: A dictionary containing various statistics about the aspect ratios of the triangular faces in the mesh.
    """
    
    # Translate trimesh mesh to vtk style mesh.
    points = vtk.vtkPoints()
    cells = vtk.vtkCellArray()

    for vertex in mesh.vertices:
        points.InsertNextPoint(vertex)

    for face in mesh.faces:
        triangle = vtk.vtkTriangle()
        for i in range(3):
            triangle.GetPointIds().SetId(i, face[i])
        cells.InsertNextCell(triangle)

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetPolys(cells)

    quality_filter = vtk.vtkMeshQuality()
    quality_filter.SetInputData(polydata)
    quality_filter.SetTriangleQualityMeasureToAspectRatio() # The saver
    quality_filter.Update()

    aspect_ratios = []
    for i in range(quality_filter.GetOutput().GetNumberOfCells()):
        aspect_ratios.append(quality_filter.GetOutput().GetCellData().GetArray("Quality").GetValue(i))

    aspect_ratios = np.array(aspect_ratios)
    return {
        'min_aspect_ratio': np.min(aspect_ratios),
        'p25_aspect_ratio': np.percentile(aspect_ratios, 25),
        'median_aspect_ratio': np.median(aspect_ratios),
        'p75_aspect_ratio': np.percentile(aspect_ratios, 75),
        'p90_aspect_ratio': np.percentile(aspect_ratios, 90),
        'p95_aspect_ratio': np.percentile(aspect_ratios, 95),
        'max_aspect_ratio': np.max(aspect_ratios),
        'ave_aspect_ratio': np.mean(aspect_ratios),
    }

In [17]:
def calculate_dihedral_angles(mesh):
    """
    Calculate the dihedral angles between adjacent faces in the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    dict: A dictionary containing various statistics about the dihedral angles.
        If the array of dihedral angles is empty, returns `None` for all values.
    """
    dihedral_angles = mesh.face_adjacency_angles

    if dihedral_angles.size == 0:
        # Return None or any indicative value if no angles are available.
        # I put it because there were some problems while running the loop the first time.
        return {
            'min_dihedral_angle': None,
            'p25_dihedral_angle': None,
            'median_dihedral_angle': None,
            'p75_dihedral_angle': None,
            'p90_dihedral_angle': None,
            'p95_dihedral_angle': None,
            'max_dihedral_angle': None,
            'ave_dihedral_angle': None
        }

    return {
        'min_dihedral_angle': np.min(dihedral_angles),
        'p25_dihedral_angle': np.percentile(dihedral_angles, 25),
        'median_dihedral_angle': np.median(dihedral_angles),
        'p75_dihedral_angle': np.percentile(dihedral_angles, 75),
        'p90_dihedral_angle': np.percentile(dihedral_angles, 90),
        'p95_dihedral_angle': np.percentile(dihedral_angles, 95),
        'max_dihedral_angle': np.max(dihedral_angles),
        'ave_dihedral_angle': np.mean(dihedral_angles)
    }

In [18]:
def calculate_area_stats(mesh):
    """
    Calculate various statistical measures of the face areas in the mesh.

    This method computes statistics such as total area, minimum area, percentiles, and average area of the faces in the mesh.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    dict: A dictionary containing various statistics about the face areas.
    """
    areas = mesh.area_faces
    return {
        'total_area': np.sum(areas),
        'min_area': np.min(areas),
        'p25_area': np.percentile(areas, 25),
        'median_area': np.median(areas),
        'p75_area': np.percentile(areas, 75),
        'p90_area': np.percentile(areas, 90),
        'p95_area': np.percentile(areas, 95),
        'max_area': np.max(areas),
        'ave_area': np.mean(areas),
    }

In [19]:
def check_solid(mesh):
    """
    Check if the mesh is solid (watertight).

    A solid (watertight) mesh has no holes and encloses a volume.

    Parameters:
    mesh (trimesh.Trimesh): The input mesh.

    Returns:
    bool: True if the mesh is solid (watertight), False otherwise.
    """
    return mesh.is_watertight

In [20]:
def extract_features_from_mesh(mesh):
    """
    Extracts all features by calling the corresponding methods.

    Parameters:
    mesh (trimesh.Trimesh): The mesh to analyze.

    Returns:
    list: A list of all extracted features.
    """
    num_vertices = calculate_num_vertices(mesh)
    num_faces = calculate_num_faces(mesh)
    num_geometrical_degenerated_faces = calculate_num_geometrical_degenerated_faces(mesh)
    num_combinatorial_degenerated_faces = calculate_num_combinatorial_degenerated_faces(mesh)
    num_duplicated_faces = calculate_num_duplicated_faces(mesh)
    num_connected_components = calculate_num_connected_components(mesh)
    num_boundary_edges = calculate_num_boundary_edges(mesh)
    euler_characteristic = calculate_euler_characteristic(mesh)
    num_self_intersections = calculate_num_self_intersections(mesh)
    num_coplanar_intersecting_faces = calculate_num_coplanar_intersecting_faces(mesh)
    vertex_manifold = check_vertex_manifold(mesh)
    edge_manifold = check_edge_manifold(mesh)
    oriented = check_oriented(mesh)
    valence_stats = calculate_vertex_valence(mesh)
    aspect_ratio_stats = calculate_aspect_ratios_vtk(mesh)
    dihedral_angle_stats = calculate_dihedral_angles(mesh)
    area_stats = calculate_area_stats(mesh)
    solid = check_solid(mesh)

    features = [
        num_vertices, num_faces, num_geometrical_degenerated_faces,
        num_combinatorial_degenerated_faces, num_connected_components,
        num_boundary_edges, num_duplicated_faces, euler_characteristic,
        num_self_intersections, num_coplanar_intersecting_faces,
        vertex_manifold, edge_manifold, oriented,
        area_stats['total_area'], area_stats['min_area'], area_stats['p25_area'], area_stats['median_area'],
        area_stats['p75_area'], area_stats['p90_area'], area_stats['p95_area'], area_stats['max_area'],
        valence_stats['min_valance'], valence_stats['p25_valance'], valence_stats['median_valance'],
        valence_stats['p75_valance'], valence_stats['p90_valance'], valence_stats['p95_valance'],
        valence_stats['max_valance'],
        dihedral_angle_stats['min_dihedral_angle'], dihedral_angle_stats['p25_dihedral_angle'], dihedral_angle_stats['median_dihedral_angle'],
        dihedral_angle_stats['p75_dihedral_angle'], dihedral_angle_stats['p90_dihedral_angle'], dihedral_angle_stats['p95_dihedral_angle'],
        dihedral_angle_stats['max_dihedral_angle'],
        aspect_ratio_stats['min_aspect_ratio'], aspect_ratio_stats['p25_aspect_ratio'], aspect_ratio_stats['median_aspect_ratio'],
        aspect_ratio_stats['p75_aspect_ratio'], aspect_ratio_stats['p90_aspect_ratio'], aspect_ratio_stats['p95_aspect_ratio'],
        aspect_ratio_stats['max_aspect_ratio'], solid, 
        area_stats['ave_area'], valence_stats['ave_valance'], dihedral_angle_stats['ave_dihedral_angle'], aspect_ratio_stats['ave_aspect_ratio']
    ]

    return features

In [21]:
def process_folder(folder_path):
    """
    Processes all 3D mesh files in a specified folder, extracting features from each mesh.

    Parameters:
    folder_path (str): Path to the folder containing .obj or .stl files.

    Returns:
    pandas.DataFrame: A DataFrame containing the extracted features for each mesh file.
    """
    data = []
    logging.info(f"Processing folder: {folder_path}")
    for filename in os.listdir(folder_path):
        if filename.endswith('.obj') or filename.endswith('.stl'):
            file_path = os.path.join(folder_path, filename)
            try:
                logging.info(f"Processing file: {filename}")
                mesh = trimesh.load(file_path)
                features = extract_features_from_mesh(mesh)
                file_id = os.path.splitext(filename)[0]  # Remove the extension
                data.append([file_id] + features)
            except Exception as e:
                logging.error(f"Error processing file {filename}: {e}")
    
    columns = [
        'file_id', 'num_vertices', 'num_faces',
        'num_geometrical_degenerated_faces', 'num_combinatorial_degenerated_faces',
        'num_connected_components', 'num_boundary_edges', 'num_duplicated_faces',
        'euler_characteristic', 'num_self_intersections', 'num_coplanar_intersecting_faces',
        'vertex_manifold', 'edge_manifold', 'oriented', 'total_area',
        'min_area', 'p25_area', 'median_area', 'p75_area', 'p90_area',
        'p95_area', 'max_area', 'min_valance', 'p25_valance', 'median_valance',
        'p75_valance', 'p90_valance', 'p95_valance', 'max_valance',
        'min_dihedral_angle', 'p25_dihedral_angle', 'median_dihedral_angle',
        'p75_dihedral_angle', 'p90_dihedral_angle', 'p95_dihedral_angle',
        'max_dihedral_angle', 'min_aspect_ratio', 'p25_aspect_ratio',
        'median_aspect_ratio', 'p75_aspect_ratio', 'p90_aspect_ratio',
        'p95_aspect_ratio', 'max_aspect_ratio', 'solid', 'ave_area',
        'ave_valance', 'ave_dihedral_angle', 'ave_aspect_ratio'
    ]

    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
input_folder_path = './../../data/simplified_output/'
output_folder_path='./../../data/csv_data/feature_extracted_simplified_meshes/'

In [23]:
run_folder_names = ['output_t0.0_r0.5_p2000', 'output_t0.1_r0.5_p2000', 'output_t0.1_r0.9_p2000', 'output_t0.3_r0.9_p2000']

In [24]:
log_file = 'process_log.log'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [25]:
for name in run_folder_names:
    geom_data = process_folder(input_folder_path + name)
    geom_data.to_csv(output_folder_path + name + '_geom.csv', index=False)
    del geom_data